# INF161 Project - Part 2
Notes about data:
- Bruker
    - Lots of NaN/None values
    - Should change sexes to binary
- Film
    - Seems like duplicate indexes.
    - Title contains release year
    - Genres should be split to sepearate columns with a binary value
- Rangering
    - No column names, adding BrukerID::FilmID::Rangering::Tidstempel
    - Must convert all ratings to the (0-5) scale
    - Timestamp shuld be datetime format

New features to dataset:
- Extracted release year from title
- TODO make binary value of timnestamp before and after dinner. I think users look at different content based on that factor.

In [56]:
import datetime
import json
import re
import numpy as np
from io import StringIO
import os
import pandas as pd

In [57]:
#BrukerDF
bruker_df = pd.read_json(('raw_data/bruker.json'), orient='split')

#print(bruker_df)

      BrukerID Kjonn  Alder  Jobb    Postkode
0            0  None   45.0   6.0       92103
1            1     M   50.0  16.0  55405-2546
2            2     M   18.0  20.0       44089
3            3     M    NaN   1.0       33304
4            4     M   35.0   6.0       48105
...        ...   ...    ...   ...         ...
6035      6036     M   45.0   0.0       61821
6036      6037     F    NaN   NaN        None
6037      6038  None   25.0  16.0       33301
6038      6039     M   35.0  14.0       92075
6039      6040     M   25.0   6.0       68131

[6040 rows x 5 columns]


In [58]:
#FilmDF
film_df = pd.read_excel('raw_data/film.xlsx', sheet_name="film", usecols="B:D")

# No duplicates
#print(film_df[film_df.duplicated()])

old_titles = film_df['Tittel'].to_numpy()
new_titles = []
year_realesed = []

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

for t in old_titles:
    year_realesed.append(t[-6:][1:-1])
    new_titles.append(t[:-7])

film_df['Tittel']=new_titles
film_df['Aar']=year_realesed
film_df['Aar']=pd.to_datetime(film_df.Aar, format='%Y')

#Split into binary genres
columns = ["FilmID","Tittel","Aar","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]

old = film_df['Sjanger'].to_numpy()

new = pd.DataFrame(index = range(0, len(film_df)),columns=columns).fillna(0)
new['FilmID']=film_df['FilmID']
new['Tittel']=film_df['Tittel']
new['Aar']=film_df['Aar']

for genre in columns[3:]:
    res = film_df.loc[film_df.Sjanger.str.contains(genre)]
    new.loc[film_df['FilmID'].isin(res['FilmID']), genre]=1


film_df=new
#print(film_df)

In [59]:
#RangeringDF
rangering_df = pd.read_csv('raw_data/rangering.dat', sep="::", engine='python')
# No duplicates
#print(rangering_df[rangering_df.duplicated()])

#For testing
#rangering_df = rangering_df.head(10)

In [60]:
rangering_df['Tidstempel'] = pd.to_datetime(rangering_df['Tidstempel'],unit='s')


date = datetime.datetime(2000, 7, 31)
def ten_to_five(datetime,rank):
    if datetime < date:
        return np.math.ceil(rank/2)
    else:
        return rank
result = [ten_to_five(row[0],row[1]) for row in zip(rangering_df['Tidstempel'], rangering_df['Rangering'])]
rangering_df['Rangering']=result
#print(rangering_df)

In [61]:
# creating directory and placing files.

path = 'clean_data'
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

# Write to files
bruker_df.to_csv(path+'/bruker.csv',index=False)
film_df.to_csv(path+'/film.csv',index=False)
rangering_df.to_csv(path+'/rangering.csv',index=False)

Creation of the directory clean_data failed
